# Organize product delivery to customers using Decision Optimization

This tutorial shows how to set up Decision Optimization engines and build a constraint programming model to efficiently organize product delivery to customers using a single truck. All data and instructions that you need to model and solve this problem are included. 

>This notebook is part of **[Prescriptive Analytics for Python](https://rawgit.com/IBMDecisionOptimization/docplex-doc/master/docs/index.html)**.

> When you finish this tutorial, you'll have a foundational knowledge of Prescriptive Analytics.

>This notebook requires a valid subscription to [Decision Optimization on Cloud](https://developer.ibm.com/docloud).

Some familiarity with Python is recommended. This notebook runs on Python 2.
 

## Table of contents
* [Describe the business problem](#describe-problem)
* [How Decision Optimization can help](#do-help) 
* [Use Decision Optimization to create and solve the model](#do-model-create-solve)
   1. [Download the DOcplex library](#Download-the-library)<br>
   2. [Set up the prescriptive engine](#Set-up-the-prescriptive-engine)<br>
   3. [Model the data](#Model-the-data)<br>
   4. [Set up the prescriptive model](#Set-up-the-prescriptive-model)<br>
      4.1 [Prepare data for modeling](#Prepare-data-for-modeling)<br>
      4.2 [Create the model](#Create-model)<br>
      4.3 [Define the decision variables](#Define-the-decision-variables)<br>
      4.4 [Define the business constraints](#Express-the-business-constraints)<br>
      4.5 [Specify the objective](#Express-the-objective)<br>
      4.6 [Solve the model](#Solve-with-solve-service)<br>
   5. [Investigate the solution and run an example analysis](#Investigate-the-solution)<br>
* [Summary](#summary)<br> 

<a id="describe-problem"></a>
## The business problem: Deliver products to customers quickly and efficiently 

You need to deliver a number of product orders to several customers using a single truck. Each order consists of a given quantity of a  certain type of the product. The truck can be configured to handle one, two, or three different types of the product. Loading the truck with at least one product of a given type requires some specific configuration. The configuration determines the truck capacity and its loading cost.

The following conditions must be met when planning the delivery:

* The product type is an integer in {0, 1, 2}.
* You can have 7 different truck configurations that correspond to the 7 possible combinations of product types:
 - Configuration 0: all products are of type 0.
 - Configuration 1: all products are of type 1.
 - Configuration 2: all products are of type 2.
 - Configuration 3: products are of type 0 or 1.
 - Configuration 4: products are of type 0 or 2.
 - Configuration 5: products are of type 1 or 2.
 - Configuration 6: products are of type 0 or 1 or 2.
* The cost of reconfiguring the truck from configuration A to configuration B depends on A and B.
* A delivery consists of loading the truck with one or several orders for the same customer.

Your goal is to minimize the cost of configuring and loading the truck and the number of deliveries needed to deliver all the orders, the cost being the most important criterion.

<a id="do-help"></a>
## How  decision optimization can help

Prescriptive analytics (decision optimization) technology recommends actions that are based on desired outcomes. It considers specific scenarios, resources, and knowledge of past and current events. With this insight, your organization can make better decisions and have greater control over business outcomes.

Prescriptive analytics is the next step on the path to insight-based actions. It creates value through synergy with predictive analytics, which analyzes data to predict future outcomes. Prescriptive analytics takes that insight to the next level by suggesting the optimal way to handle a future situation. Organizations that act fast in dynamic conditions and make superior decisions in uncertain environments gain a strong competitive advantage.

With prescriptive analytics, you can:

* Automate the complex decisions and trade-offs to better manage your limited resources.
    
* Take advantage of a future opportunity or mitigate a future risk.
    
* Proactively update recommendations based on changing events.
    
* Meet operational goals, increase customer loyalty, prevent threats and fraud, and optimize business processes.

<a id="do-model-create-solve"></a>
## Use Decision Optimization
Perform the following steps to create and solve the model.

<a id="Download-the-library"></a>
### 1. Download the DOcplex library

DOcplex is the Decision Optimization CPLEX Modeling library for Python. This library includes two modules, Mathematical Programming Modeling (DOcplex.MP ) and Constraint Programming Modeling (DOcplex.CP).

Run the following cell to install  the library. The `real.prefix` in the code is used to ensure that the script is not running inside a virtual environment.

In [1]:
from sys import stdout
try:
    import docplex.cp
except:
    if hasattr(sys, 'real_prefix'):
        !pip install docplex
    else:
        !pip install --user docplex

<a id="Set-up-the-prescriptive-engine"></a>
### 2. Set up the prescriptive engine

To solve DOcplex models, you need access to the DOcplexcloud solve service.

* Subscribe to the [Decision Optimization on Cloud](https://developer.ibm.com/docloud) (DOcplexcloud) service.
* Get the service base URL and personal API key.
* Enter the URL and API key in the cell below, enclosed in quotation marks (""), and run the cell. 

__Note:__ For a persistent setting, create a Python configuration file *docloud_config.py* in a location that is visible to PYTHONPATH.


In [2]:
SVC_URL = "URL"
SVC_KEY = "KEY"

Import the DOcplex library.

In [3]:
from docplex.cp.model import *

<a id="Model-the-data"></a>
### 3. Model the data

Define the data. The comments in the code provide details about the data.

In [4]:
# Possible truck configurations. Each tuple is (load, cost), where: 
# load: max truck load for this configuration
# cost: cost of loading the truck in this configuration
TRUCK_CONFIGURATIONS = ((11, 2), (11, 2), (11, 2), (11, 3), (10, 3), (10, 3), (10, 4))

# List of customer orders.
# Each tuple is (customer index, volume, product type)
CUSTOMER_ORDERS = ((0, 3, 1), (0, 4, 2), (0, 3, 0), (0, 2, 1), (0, 5, 1), (0, 4, 1), (0, 11, 0),
                   (1, 4, 0), (1, 5, 0), (1, 2, 0), (1, 4, 2), (1, 7, 2), (1, 3, 2), (1, 5, 0), (1, 2, 2),
                   (2, 5, 1), (2, 6, 0), (2, 11, 2), (2, 1, 0), (2, 6, 0), (2, 3, 0))

# Transition costs between configurations.
# Tuple (A, B, TCost) where TCost is the cost of modifying the truck from configuration A to configuration B
CONFIGURATION_TRANSITION_COST = tuple_set(((0, 0,  0), (0, 1,  0), (0, 2,  0), (0, 3, 10), (0, 4, 10),
                                           (0, 5, 10), (0, 6, 15), (1, 0,  0), (1, 1,  0), (1, 2,  0),
                                           (1, 3, 10), (1, 4, 10), (1, 5, 10), (1, 6, 15), (2, 0,  0),
                                           (2, 1,  0), (2, 2,  0), (2, 3, 10), (2, 4, 10), (2, 5, 10),
                                           (2, 6, 15), (3, 0,  3), (3, 1,  3), (3, 2,  3), (3, 3,  0),
                                           (3, 4, 10), (3, 5, 10), (3, 6, 15), (4, 0,  3), (4, 1,  3),
                                           (4, 2,  3), (4, 3, 10), (4, 4,  0), (4, 5, 10), (4, 6, 15),
                                           (5, 0,  3), (5, 1,  3), (5, 2,  3), (5, 3, 10), (5, 4, 10),
                                           (5, 5,  0), (5, 6, 15), (6, 0,  3), (6, 1,  3), (6, 2,  3),
                                           (6, 3, 10), (6, 4, 10), (6, 5, 10), (6, 6,  0)
                                           ))

# Dependency between the product types and the configuration of the truck
# allowedContainerConfigs[i]: the array of all the configurations that accept products of type i
ALLOWED_CONTAINER_CONFIGS = ((0, 3, 4, 6),
                             (1, 3, 5, 6),
                             (2, 4, 5, 6))

<a id="Set-up-the-prescriptive-model"></a>
### 4. Set up the prescriptive model

<a id="Prepare-data-for-modeling"></a>
####  4.1. Prepare data for modeling

Extract data that is frequently used when modeling.

In [5]:
nbTruckConfigs = len(TRUCK_CONFIGURATIONS)
maxTruckConfigLoad = [tc[0] for tc in TRUCK_CONFIGURATIONS]
truckCost = [tc[1] for tc in TRUCK_CONFIGURATIONS]
maxLoad = max(maxTruckConfigLoad)

nbOrders = len(CUSTOMER_ORDERS)
nbCustomers = 1 + max(co[0] for co in CUSTOMER_ORDERS)
volumes = [co[1] for co in CUSTOMER_ORDERS]
productType = [co[2] for co in CUSTOMER_ORDERS]

# Maximum number of truck deliveries (estimated upper bound to be increased if there is no solution)
maxDeliveries = 15

<a id="Create-model"></a>
####  4.2  Create the model

In [6]:
mdl = CpoModel(name="trucks")

<a id="Define-the-decision-variables"></a>
####  4.3 Define the decision variables

The comments in the code provide details about the variables. 

In [7]:
# Configuration of the truck for each delivery
truckConfigs = integer_var_list(maxDeliveries, 0, nbTruckConfigs - 1, "truckConfigs")
# Association between delivery and order
where = integer_var_list(nbOrders, 0, maxDeliveries - 1, "where")
# Truck load
load = integer_var_list(maxDeliveries, 0, maxLoad, "load")
# Number of required deliveries
nbDeliveries = integer_var(0, maxDeliveries)
# Association between a customer and a delivery
customerOfDelivery = integer_var_list(maxDeliveries, 0, nbCustomers, "customerOfTruck")
# Transition cost for each delivery
transitionCost = integer_var_list(maxDeliveries - 1, 0, 1000, "transitionCost")

<a id="Express-the-business-constraints"></a>
####  4.4 Specify the business constraints

The comments in the code provide details about the constraints.

In [8]:
# transitionCost[i] = transition cost between configurations i and i+1
for i in range(1, maxDeliveries):
    auxVars = (truckConfigs[i - 1], truckConfigs[i], transitionCost[i - 1])
    mdl.add(allowed_assignments(auxVars, CONFIGURATION_TRANSITION_COST))

# Constrain the volume of orders in each truck
mdl.add(pack(load, where, volumes, nbDeliveries))
for i in range(0, maxDeliveries):
    mdl.add(load[i] <= element(truckConfigs[i], maxTruckConfigLoad))

# Relationship between the product type of an order and the configuration of its truck
for j in range(0, nbOrders):
    configOfContainer = integer_var(ALLOWED_CONTAINER_CONFIGS[productType[j]])
    mdl.add(configOfContainer == element(truckConfigs, where[j]))

# Assign only one customer per delivery
for j in range(0, nbOrders):
    mdl.add(element(customerOfDelivery, where[j]) == CUSTOMER_ORDERS[j][0])

# Unused deliveries are at the end
for j in range(1, maxDeliveries):
    mdl.add((load[j - 1] > 0) | (load[j] == 0))

# Dominance: the unused deliveries keep the last used configuration
mdl.add(load[0] > 0)
for i in range(1, maxDeliveries):
    mdl.add((load[i] > 0) | (truckConfigs[i] == truckConfigs[i - 1]))

# Dominance: regroup deliveries with the same configuration
for i in range(maxDeliveries - 2, 0, -1):
    ct = true()
    for p in range(i + 1, maxDeliveries):
        ct = (truckConfigs[p] != truckConfigs[i - 1]) & ct
    mdl.add((truckConfigs[i] == truckConfigs[i - 1]) | ct)

<a id="Express-the-objective"></a>
####  4.5 Specify the objective

The objectives are:
* Minimize the cost of configuring and loading trucks. 
* Minimize the number of deliveries. 

In [9]:
cost = sum(transitionCost) + sum(element(truckConfigs[i], truckCost) * (load[i] != 0) for i in range(maxDeliveries))
mdl.add(minimize_static_lex([cost, nbDeliveries]))

<a id="Solve-with-solve-service"></a>
#### 4.6 Solve the model

Use the Decision Optimization on Cloud solve service to solve the model.

In [10]:
# Search strategy: assign an order to a truck
mdl.set_search_phases([search_phase(where)])

print("\nSolving model....")
msol = mdl.solve(url=SVC_URL, key=SVC_KEY, TimeLimit=20, LogPeriod=3000)


Solving model....


<a id="Investigate-the-solution"></a>
### 5. Investigate the solution and run an example analysis

Run the following cell. You might need to wait a few seconds for the solution.

In [11]:
if msol.is_solution():
    print("Solution: ")
    ovals = msol.get_objective_values()
    print("   Configuration cost: {}, number of deliveries: {}".format(ovals[0], ovals[1]))
    for i in range(maxDeliveries):
        ld = msol.get_value(load[i])
        if ld > 0:
            stdout.write("   Delivery {:2d}: config={}".format(i,msol.get_value(truckConfigs[i])))
            stdout.write(", items=")
            for j in range(nbOrders):
                if (msol.get_value(where[j]) == i):
                    stdout.write(" <{}, {}, {}>".format(j, productType[j], volumes[j]))
            stdout.write('\n')
else:
    stdout.write("Solve status: {}\n".format(msol.get_solve_status()))

Solution: 
   Configuration cost: 26.0, number of deliveries: 13.0
   Delivery  0: config=1, items= <4, 1, 5>
   Delivery  1: config=1, items= <0, 1, 3> <3, 1, 2> <5, 1, 4>
   Delivery  2: config=1, items= <15, 1, 5>
   Delivery  3: config=0, items= <7, 0, 4> <8, 0, 5>
   Delivery  4: config=0, items= <18, 0, 1> <19, 0, 6>
   Delivery  5: config=0, items= <2, 0, 3>
   Delivery  6: config=0, items= <6, 0, 11>
   Delivery  7: config=0, items= <9, 0, 2> <13, 0, 5>
   Delivery  8: config=0, items= <16, 0, 6> <20, 0, 3>
   Delivery  9: config=2, items= <17, 2, 11>
   Delivery 10: config=2, items= <10, 2, 4> <12, 2, 3>
   Delivery 11: config=2, items= <1, 2, 4>
   Delivery 12: config=2, items= <11, 2, 7> <14, 2, 2>


The proposed solution specifies 26 as the minimal cost of configuring and loading trucks, and 13 as the number of deliveries. The solution suggests 12 deliveries, and a configuration for each delivery. 

<a id="summary"></a>
## Summary

You learned how to set up and use the IBM Decision Optimization CPLEX Modeling for Python to build a Constraint Programming model and solve it with IBM Decision Optimization on Cloud.

#### References
* [CPLEX Modeling for Python documentation](https://rawgit.com/IBMDecisionOptimization/docplex-doc/master/docs/index.html)
* [Decision Optimization on Cloud](https://developer.ibm.com/docloud/)
* [Decision Optimization documentation](https://dataplatform.cloud.ibm.com/docs/content/DO/DOinDSX.html)
* For help with DOcplex, or to report a defect, go [here](https://developer.ibm.com/answers/smartspace/docloud).
* Contact us at dofeedback@wwpdl.vnet.ibm.com

<div class="alert alert-block alert-info"> Note: To save resources and get the best performance please use the code below to stop the kernel before exiting your notebook.</div>

In [ ]:
%%javascript
Jupyter.notebook.session.delete();

<hr>
Copyright &copy; IBM Corp. 2017. Released as licensed Sample Materials.